# Chat

Recall the overall workflow for retrieval augmented generation (RAG):

We discussed `Document Loading` and `Splitting` as well as `Storage` and `Retrieval`.

We then showed how `Retrieval` can be used for output generation in Q+A using `RetrievalQA` chain.

In [1]:
import os
import openai
import sys

import panel as pn  # GUI
pn.extension()

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
llm_name = "gpt-3.5-turbo"

If you wish to experiment on `LangChain plus platform`:

 * Go to [langchain plus platform](https://www.langchain.plus/) and sign up
 * Create an api key from your account's settings
 * Use this api key in the code below 

In [3]:
#import os
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
#os.environ["LANGCHAIN_API_KEY"] = "..."

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = './resources/docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [6]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

2

In [7]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm.predict("Hello world!")

'Hello! How can I assist you today?'

In [8]:
# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)

# Run chain
from langchain.chains import RetrievalQA
question = "Is probability a class topic?"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

'Yes, probability is a class topic in many math and statistics courses. Thanks for asking!'

### Memory

In [9]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

### ConversationalRetrievalChain

In [10]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [11]:
question = "Is probability a class topic?"
result = qa({"question": question})

In [12]:
result['answer']

'Yes, probability is often taught as a topic in mathematics or statistics classes. It is a branch of mathematics that deals with the likelihood of events occurring.'

In [13]:
question = "why are those prerequesites needed?"
result = qa({"question": question})

In [14]:
result['answer']

'Prerequisites are needed for probability because probability involves concepts and techniques from other fields such as statistics and linear algebra. Understanding basic probability and statistics is important for analyzing and interpreting data, making predictions, and understanding uncertainty. Familiarity with linear algebra is necessary for working with matrices and vectors, which are often used in probability calculations and modeling. Having a solid foundation in these prerequisite subjects helps students grasp the core concepts and methods of probability more effectively.'

# Create a chatbot that works on your documents

In [15]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

The chatbot code has been updated a bit since filming. The GUI appearance also varies depending on the platform it is running on.

In [16]:
def load_db(file, chain_type, k):
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # define embedding
    embeddings = OpenAIEmbeddings()
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0), 
        chain_type=chain_type, 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa 


In [21]:
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "./resources/docs/cs229_lectures/MachineLearning-Lecture01.pdf"
        self.qa = load_db(self.loaded_file,"stuff", 4)
    
    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return 
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return 


### Create a chatbot

In [23]:
cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp) 

jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
dashboard

/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/panel/viewable.py:298: ParamFutureWarning: Parameter 'object' on <class 'panel.pane.base.PaneBase'> is being given a valid parameter reference <function _param_bind.<locals>.wrapped at 0x174e401f0> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <function _param_bind.<locals>.wrapped at 0x174e401f0> will be resolved to its underlying value WidgetBox(scroll=True)
    [0] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600). Please explicitly set allow_ref on the Parameter definition to declare whether references should be resolved or not.
  super().__init__(**params)
/Users/miesner.jacob/.pyenv/versions/3.8.6/lib/python3.8/site-packages/panel/viewable.py:298: ParamFutureWarning: Parameter 'object' on <class 'panel.pane.base.PaneBase'> is being given a valid parameter reference <bound method cbfs.get_sources of cbfs(answer='', chat_history=[], db_quer

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, height=300, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False)
            [1] Divider()
            [2] ParamMethod(method, _pane=Str, defer_load=False)
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False)
            [1] Divider()
        [3] Column
            [0] Row
                [0] FileInput(accept='.pdf')
                [1] Button(button_type='primary', name='Load DB')
                [2] ParamFunction(function, _pane=Markdown, defer_load=False)
            [1] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)
            [2] Divider()
            [3] Row
                [0] Image(str, width=400)

Feel free to copy this code and modify it to add your own features. You can try alternate memory and retriever models by changing the configuration in `load_db` function and the `convchain` method. [Panel](https://panel.holoviz.org/) and [Param](https://param.holoviz.org/) have many useful features and widgets you can use to extend the GUI.


## Acknowledgments

Panel based chatbot inspired by Sophia Yang, [github](https://github.com/sophiamyang/tutorials-LangChain)

## Experiment on your own

In [34]:
# Build Reference data
persist_directory = 'resources/docs/chroma'
llm = ChatOpenAI(model_name=llm_name, temperature=0)
embedding = OpenAIEmbeddings()

machine_learning_texts = [
    "Machine Learning, a branch of artificial intelligence, focuses on the development of algorithms that enable computers to learn from and make predictions or decisions based on data. Unlike traditional programming, where instructions are explicitly given, ML algorithms improve automatically through experience.",
    "Supervised learning, a primary method within Machine Learning, involves training an algorithm on a labeled dataset, where the correct output is known, enabling the model to learn over time to predict the output when given new data.",
    "Unsupervised learning, another core technique in Machine Learning, deals with input data without labeled responses. The system tries to learn the patterns and the structure from such data by itself, often used for clustering or anomaly detection.",
    "Reinforcement learning is a type of Machine Learning where an algorithm learns to perform an action from experience by making decisions in a game-like scenario. It optimizes its decision-making process based on the rewards received for actions taken.",
    "Deep Learning, a subset of Machine Learning, utilizes neural networks with many layers. These algorithms are particularly powerful in handling vast amounts of data and are behind many advances in image and speech recognition, language translation, and autonomous vehicles.",
    "Feature engineering is a crucial step in the Machine Learning process, involving the selection, modification, and creation of input variables or features that are used to train the model. It significantly impacts the performance of ML algorithms.",
    "Overfitting in Machine Learning occurs when a model learns the detail and noise in the training data to the extent that it performs poorly on new data. Techniques such as cross-validation, regularization, and pruning are used to avoid this issue.",
    "Machine Learning models require large volumes of data for training. However, the quality and relevance of the data are equally important, as biased or incomplete data can lead to inaccurate models that perpetuate biases or fail to perform as intended.",
    "The ethical implications of Machine Learning are a growing concern, as the technology can inadvertently reinforce existing biases, infringe on privacy, or be used in manipulative ways. Ensuring fairness, accountability, and transparency in ML models is crucial.",
    "The future of Machine Learning promises advancements in personalized medicine, environmental protection, and automated systems. However, it also poses challenges in job displacement, security, and ensuring ethical use of the technology.",
]

machine_learning_metadatas = [
    {"summary": "Introduction to ML", "category": "Basics"},
    {"summary": "Supervised Learning Overview", "category": "Learning Types"},
    {"summary": "Unsupervised Learning Explained", "category": "Learning Types"},
    {"summary": "Basics of Reinforcement Learning", "category": "Learning Types"},
    {"summary": "Deep Learning and Neural Networks", "category": "Advanced Topics"},
    {"summary": "Feature Engineering Techniques", "category": "Model Preparation"},
    {"summary": "Handling Overfitting in Models", "category": "Model Evaluation"},
    {"summary": "Importance of Data Quality", "category": "Data Preparation"},
    {"summary": "Ethical Considerations in ML", "category": "Ethics and Future"},
    {"summary": "Future Trends in Machine Learning", "category": "Ethics and Future"}
]


ml_db = Chroma.from_texts(
    texts=machine_learning_texts,
    metadatas=machine_learning_metadatas,
    embedding=embedding)

In [35]:
# Set up Conversation Memory

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [36]:
# Create Conversation Chain

from langchain.chains import ConversationalRetrievalChain
retriever=ml_db.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [37]:
# Have a conversation!

question = "What are some topics referenced in this text?"
result = qa({"question": question})

print(result['answer'])

question = "What would be the best topic for a beginner in ML to start with?"
result = qa({"question": question})

print('\n')
print(result['answer'])

Some topics referenced in this text are:
1. Ethical implications of Machine Learning
2. Fairness, accountability, and transparency in ML models
3. Advancements in personalized medicine
4. Environmental protection
5. Automated systems
6. Job displacement
7. Security
8. Ethical use of Machine Learning
9. Data quality and relevance
10. Biased or incomplete data
11. Deep Learning
12. Neural networks with many layers
13. Image and speech recognition
14. Language translation
15. Autonomous vehicles


For a beginner in Machine Learning, a suitable topic to start with would be supervised learning. Supervised learning is a fundamental concept in Machine Learning and provides a solid foundation for understanding the principles and techniques used in this field. It involves training a model on a labeled dataset, where the correct output is known, and then using that model to make predictions on new, unseen data. This topic will introduce you to key concepts such as data preprocessing, model train